In [1]:
import os
import glob
import numpy

from pypeit.metadata import PypeItMetaData

In [2]:
#file_list = glob.glob(os.path.join(os.environ['PYPEIT_DEV'],
#                                   'RAW_DATA/Keck_LRIS_red/multi*/*.fits*'))
#file_list = glob.glob(os.path.join(os.environ['PYPEIT_DEV'],
#                                   'RAW_DATA/Keck_LRIS_red/multi_1200_9000_d680/*.fits*'))
file_list = glob.glob(os.path.join(os.environ['PYPEIT_DEV'],
                                   'RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/*.fits*'))

Construct the metadata just based on the headers

In [3]:
fitstbl = PypeItMetaData('keck_lris_red', file_list=file_list, background_index=True)

[WARNING] :: metadata.py 184 _build() - Columns are missing.
[WARNING] :: metadata.py 229 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 255 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2017.fits.gz
[WARNING] :: metadata.py 229 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 255 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2057.fits.gz
[WARNING] :: metadata.py 229 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 255 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2058.fits.gz
[WARNING] :: metadata.py 229 _build() - ut keyword not in heade

[WARNING] :: metadata.py 229 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 255 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2095.fits.gz
[WARNING] :: metadata.py 229 _build() - ut keyword not in header. Setting to None
[INFO]    :: metadata.py 255 _build() - Successfully loaded headers for file:
             /Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560/r170320_2096.fits.gz
[INFO]    :: metadata.py 258 _build() - Headers loaded for 27 files successfully


Add the frame types as we've always done

In [4]:
_ = fitstbl.get_frame_types(flag_unknown=True)

[INFO]    :: metadata.py 866 get_frame_types() - Typing files
[WARNING] :: keck_lris.py 130 check_frame_type() - Cannot determine if frames are of type standard.
[INFO]    :: metadata.py 935 get_frame_types() - Typing completed!


Find the unique configurations.  The unique configurations are found by matching data in the headers between frames.  The keywords that are matched is set by the `configuration_keys` in each spectrograph class.

In [5]:
cfgs = fitstbl.unique_configurations(ignore_frames=['bias', 'dark'])

[INFO]    :: metadata.py 485 unique_configurations() - Using metadata to determine unique configurations.
[INFO]    :: metadata.py 493 unique_configurations() - Unique configurations ignore frames with type: ['bias', 'dark']
[INFO]    :: metadata.py 540 unique_configurations() - Found 1 unique configurations.


In [6]:
cfgs

{'A': {'dispname': '400/8500', 'dichroic': '560', 'decker': 'c17_60L.'}}

Once you have the unique configurations, the next step is to identify each frame with a configuration.  The configuration is added to the table in the `configuration` column.  The end result of `unique_configurations` and `set_configurations` can be done in a single step.

In [7]:
fitstbl.set_configurations(cfgs)

Then you identify each frame within a given configuration as being part of a calibration group.  Calibration frames can be assigned to multiple calibration groups, but science frames can only be assigned to a single calibration group.  I.e., the calibration group of a science frame selects which calibration frames to use for it.  At the moment, some frame types can be assigned to calibrations independent of their configuration.  These are defined by the `global_frames` keyword below.  The calibration group is currently an integer or list of integers in the `calib` column of the table.

At the moment, frames are assigned to a calibration group just based on the configuration, but the logic will become more complicated.  I.e., impose a time limit on when arcs should be used with given science frames.

In [8]:
fitstbl.set_calibration_groups(global_frames=['bias', 'dark'])

In [9]:
fitstbl

index,directory,filename,instrume,idname,time,utc,ut,ra,dec,airmass,binning,decker,dichroic,target,exptime,hatch,dispname,dispangle,wavecen,spectrograph,lampstat01,lampstat02,lampstat03,lampstat04,lampstat05,lampstat06,lampstat07,lampstat08,lampstat09,lampstat10,lampstat11,lampstat12,filter1,NAXIS01,CCDGEOM,CCDNAME01,NAXIS02,NAXIS03,CCDNAME02,NAXIS04,index_bg,frametype,framebit,configuration,calib,calibbit
int64,str97,str20,str13,str4,float64,str11,str4,str11,str11,float64,str3,str8,str3,str8,int64,str6,str8,float64,float64,str4,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str5,int64,str27,str4,int64,int64,str4,int64,object,str15,int16,str4,str4,int64
0,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2017.fits.gz,keck_lris_red,dark,57832.040239,00:57:56.64,None,15:20:00.00,+00:00:00.0,13.37481353,"2,2",c17_60L.,560,unknown,1,closed,400/8500,22.13431168,6429.00585938,LRIS,on,on,on,on,on,on,on,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,arc,1,A,0,1
1,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2057.fits.gz,keck_lris_red,dark,57832.09924,02:22:54.30,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",c17_60L.,560,unknown,7,open,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,on,on,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,"trace,pixelflat",144,A,0,1
2,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2058.fits.gz,keck_lris_red,dark,57832.099994,02:23:58.90,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",c17_60L.,560,unknown,7,open,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,on,on,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,"trace,pixelflat",144,A,0,1
3,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2059.fits.gz,keck_lris_red,dark,57832.100729,02:25:03.00,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",c17_60L.,560,unknown,7,open,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,on,on,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,"trace,pixelflat",144,A,0,1
4,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2067.fits.gz,keck_lris_red,dark,57832.134889,03:14:14.36,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,None,0,1
5,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2068.fits.gz,keck_lris_red,dark,57832.135557,03:15:12.13,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,None,0,1
6,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2069.fits.gz,keck_lris_red,dark,57832.136243,03:16:11.43,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,None,0,1
7,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2070.fits.gz,keck_lris_red,dark,57832.136922,03:17:10.02,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,None,0,1
8,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d

This matches calibration frames to science frames using the existing approach.

In [10]:
fitstbl.match_to_science(fitstbl.par['calibrations'], fitstbl.par['rdx']['calwin'], fitstbl.par['fluxcalib'], setup=True)

[INFO]    :: metadata.py 1054 match_to_science() - Matching calibrations to Science frames
[INFO]    :: metadata.py 1073 match_to_science() - ==================================================
[INFO]    :: metadata.py 1074 match_to_science() - Matching calibrations to frame: c17_60L.
[INFO]    :: metadata.py 1130 match_to_science() -   Found 1 arc frame for c17_60L. (1 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 3 trace frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 0 standard frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 3 pixelflat frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 10 bias frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1114 match_to_science() - No matching criteria for pinhole frames
[INFO]    :: metadata.py 1130 match_to_science() -   Found 0 pinhole frame for c17_60L. (0 required)
[INFO]    :: metadata.p

[INFO]    :: metadata.py 1073 match_to_science() - ==================================================
[INFO]    :: metadata.py 1074 match_to_science() - Matching calibrations to frame: c17_60L.
[INFO]    :: metadata.py 1130 match_to_science() -   Found 1 arc frame for c17_60L. (1 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 3 trace frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 0 standard frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 3 pixelflat frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1130 match_to_science() -   Found 10 bias frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1114 match_to_science() - No matching criteria for pinhole frames
[INFO]    :: metadata.py 1130 match_to_science() -   Found 0 pinhole frame for c17_60L. (0 required)
[INFO]    :: metadata.py 1088 match_to_science() -   Dark frames not required.  Not matching...
[INFO]    :: metad

The following is just a check to compare what one would get using the calib group to assign the science ID.  In the test cases with a single configuration the two are identical.

In [11]:
fitstbl.calib_to_science()

In [12]:
fitstbl

index,directory,filename,instrume,idname,time,utc,ut,ra,dec,airmass,binning,decker,dichroic,target,exptime,hatch,dispname,dispangle,wavecen,spectrograph,lampstat01,lampstat02,lampstat03,lampstat04,lampstat05,lampstat06,lampstat07,lampstat08,lampstat09,lampstat10,lampstat11,lampstat12,filter1,NAXIS01,CCDGEOM,CCDNAME01,NAXIS02,NAXIS03,CCDNAME02,NAXIS04,index_bg,frametype,framebit,configuration,calib,calibbit,failures,sci_ID
int64,str97,str20,str13,str4,float64,str11,str4,str11,str11,float64,str3,str8,str3,str8,int64,str6,str8,float64,float64,str4,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str3,str5,int64,str27,str4,int64,int64,str4,int64,object,str15,int16,str4,str4,int64,bool,int64
0,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2017.fits.gz,keck_lris_red,dark,57832.040239,00:57:56.64,None,15:20:00.00,+00:00:00.0,13.37481353,"2,2",c17_60L.,560,unknown,1,closed,400/8500,22.13431168,6429.00585938,LRIS,on,on,on,on,on,on,on,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,arc,1,A,0,1,False,8191
1,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2057.fits.gz,keck_lris_red,dark,57832.09924,02:22:54.30,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",c17_60L.,560,unknown,7,open,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,on,on,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,"trace,pixelflat",144,A,0,1,False,8191
2,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2058.fits.gz,keck_lris_red,dark,57832.099994,02:23:58.90,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",c17_60L.,560,unknown,7,open,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,on,on,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,"trace,pixelflat",144,A,0,1,False,8191
3,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2059.fits.gz,keck_lris_red,dark,57832.100729,02:25:03.00,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",c17_60L.,560,unknown,7,open,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,on,on,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,"trace,pixelflat",144,A,0,1,False,8191
4,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2067.fits.gz,keck_lris_red,dark,57832.134889,03:14:14.36,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,None,0,1,False,8191
5,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2068.fits.gz,keck_lris_red,dark,57832.135557,03:15:12.13,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,None,0,1,False,8191
6,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2069.fits.gz,keck_lris_red,dark,57832.136243,03:16:11.43,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,None,0,1,False,8191
7,/Users/westfall/Work/packages/PypeIt-development-suite/RAW_DATA/Keck_LRIS_red/multi_400_8500_d560,r170320_2070.fits.gz,keck_lris_red,dark,57832.136922,03:17:10.02,None,15:20:00.00,+45:00:00.0,1.41309236,"2,2",sp15l.fi,560,unknown,0,closed,400/8500,22.13544083,6429.89794922,LRIS,off,off,off,off,off,off,off,off,off,off,off,off,Clear,2,LBNL Thick High-Resistivity,19-3,2,2,19-2,2,None,bias,2,N